In [1]:
import ipywidgets as widgets
import nanohub.uidl.teleport as t
from nanohub.uidl.rappture import RapptureBuilder
from nanohub.uidl.material import MaterialBuilder
from nanohub.uidl.material import MaterialContent, MaterialComponents
from nanohub.uidl.plotly import PlotlyBuilder
from nanohub.uidl.teleport import TeleportElement, TeleportContent, TeleportComponent
from nanohub.uidl.app import AppBuilder, FormHelper
from nanohub.uidl.ipywidgets import buildWidget

component = "MyComponent"

Project = t.TeleportProject(component)
Component = Project.root

Component.addStateVariable("open_params", {"type":"boolean", "defaultValue": True})
Component.addPropVariable("onChange", {"type":"func", "defaultValue": "(e)=>{return e;}"})
Component.addPropVariable("onClick", {"type":"func", "defaultValue": "(e)=>{return ''}"})
Component.addPropVariable("onSubmit", {"type":"func", "defaultValue": "(e)=>{return ''; }"})

SETTINGS = {
    "mat" :    {
        "type":"Select",
        "default_value": "Au-Gold", 
        "units" : "", 
        "options" : {"Au-Gold":"Au-Gold", "Ag-Silver":"Ag-Silver", "Constant":"Constant" },
        "label" : "Particle Composition",
        "description" : '''Choose a material for the particles. The material selection determines the dielectric 
            properties of the nanoparticle. If 'Constant' is selected the material is assumed to be free of loss 
            and the dielectric constant is assumed to be independent of wavelength. ''',
    },
    "cindex" : {
        "type":"Number",
        "default_value": 1.4, 
        "units" : "", 
        "options":{},
        "label" : "Particle Refractive Index", 
        "description" : '''Enter the refractive index of the particle.''', 
        "restrictions" : {"mat":"Constant"}
    },
    "refmed" : {
        "type":"Number",
        "default_value": 1.0, 
        "units" : "", 
        "options":{},
        "label" : "Surrounding Medium Refractive Index",
        "description" : '''Enter the refractive index of the surrounding medium''',
    },
    "radius" : {
        "type":"Number",
        "default_value": 20, 
        "units" : "nm", 
        "options":{},
        "label" : "Radius of particle",
        "description" : '''Radius of particle''',
    },
    "bwavel" : {
        "type":"Number",
        "default_value": 300, 
        "units" : "nm", 
        "options":{},
        "label" : "Beginning wavelength",
        "description" : '''Beginning wavelength''',
    },   
    "ewavel" : {
        "type":"Number",
        "default_value": 1000, 
        "units" : "nm", 
        "options":{},
        "label" : "Ending wavelength",
        "description" : '''Ending wavelength''',
    },
}
LAYOUT = {
    'input': {
        'type': 'group',
        'id': '',
        'label': '',
        'layout': '',
        'children': [
            { 'type': 'choice', 'id': 'mat', 'label': 'Particle Composition'},
            { 'type': 'number', 'id': 'cindex', 'label': 'Particle Refractive Index', 'enable': [
                {'operand': 'mat', 'operator': '==', 'value': "Constant"}
            ]},
            { 'type': 'number', 'id': 'refmed', 'label': 'Surrounding Medium Refractive Index'},
            { 'type': 'number', 'id': 'radius', 'label': 'Radius of particle'},
            {'type': 'group', 'label': 'wavelength range', 'layout': 'horizontal', 'children': [
                {'type': 'number', 'id': 'bwavel', 'label': 'Beginning wavelength', 'enable': None},
                {'type': 'number', 'id': 'ewavel', 'label': 'Ending wavelength', 'enable': None}
            ]}
        ]
    }
}
Settings = AppBuilder.Settings(
    Project,
    Component, 
    SETTINGS,
    url="",
    toolname = "",
    layout = LAYOUT['input']
)

Settings.content.events["onError"]=[
]


Settings.content.events["onChange"]=[
    { "type": "stateChange", "modifies": "parameters","newState": '$Object.assign({},self.state.parameters, {[e.id]:e.value})'},
    { "type": "propCall2", "calls": "onChange","args": ["e"] },
]

Settings.content.events["click"]=[
    { "type": "propCall2", "calls": "onClick","args": ["e"] },
]

Settings.content.events["submit"] = [
    { "type": "propCall2", "calls": "onSubmit","args": ["e"] },
]

ParametersPanel = MaterialBuilder.ExpansionPanel(
    title="Parameters", 
    expanded={
      "type": "dynamic",
      "content": {
        "referenceType": "state",
        "id": "open_params"
      }    
    }, 
    content=[Settings]
)
ParametersPanel.content.events['change'] = [{ "type": "stateChange", "modifies": "open_params","newState": "$toggle"}]

Component.addNode(ParametersPanel)


In [2]:
MyComponentWidget = buildWidget(Project);


<IPython.core.display.Javascript object>

In [3]:
out = widgets.Output()                
def click(a=None, **kwargs):
    global out
    with out:
        print(a,kwargs)
        
v = MyComponentWidget(onChange=click, onClick=click, parameters = {'mat': 'Au-Gold',
 'cindex': 1.1,
 'refmed': 31,
 'radius': 11,
 'bwavel': 301,
 'ewavel': 1001}
)
v

MyComponentWidget(parameters={'mat': 'Au-Gold', 'cindex': 1.1, 'refmed': 31, 'radius': 11, 'bwavel': 301, 'ewa…

In [4]:
display(out)

Output()

In [5]:
v.observe(click)

In [6]:
v.parameters = {'mat': 'Constant',
 'cindex': 1.4,
 'refmed': 32,
 'radius': 10,
 'bwavel': 310,
 'ewavel': 1000
}

In [7]:
v.parameters

{'mat': 'Constant',
 'cindex': 1.4,
 'refmed': 32,
 'radius': 10,
 'bwavel': 310,
 'ewavel': 1000}

In [8]:
v.parameters = {'bwavel': '306'}

In [9]:
v.parameters

{'mat': 'Constant',
 'cindex': 1.4,
 'refmed': 32,
 'radius': 10,
 'bwavel': '306',
 'ewavel': 1000}

In [10]:
v.test_number = "1.33"


In [11]:
v.test_number

'1.33'